# Compare Models

Create object to automated model comparisons

In [1]:
import arithmetic_datasets as ad

from tensorflow.keras.layers import Input, Lambda, Dense, concatenate
from tensorflow.keras.models import Model, Sequential

import matplotlib.pyplot as plt
import random
import numpy as np

from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
import tensorflow as tf
import random
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Metric

from tensorflow.keras.callbacks import Callback

from tensorflow.keras.models import load_model

import os

## Callback Directory Filepath

In [2]:
#WIP
cb_filepath = "checkpoints"
if not os.path.exists(cb_filepath):
    os.makedirs(cb_filepath)  

## Save Model Callback

In [3]:
class SaveModelCallback(Callback):
    def __init__(self, filepath,  n=100):
        super(SaveModelCallback, self).__init__()
        self.num_epochs = 100
        self.filepath = filepath
    
    def on_epoch_end(self, epoch, logs = None):
        super(SaveModelCallback, self).on_epoch_end(epoch, logs)
        if epoch % 100 == 0:
            self.model.save(f"{self.filepath}/{self.model.name}/{epoch}.model")
            print(f"model saved: {self.filepath}/{self.model.name}/{epoch}.model")

In [4]:
smc = SaveModelCallback(cb_filepath)

### ModelComparator

```
mc = ModelComparator()
mc.add_model(name, model)
mc.add_training_set(name, data)

mc.train_models(epochs, batches, **kwargs)

mc.compare_min(metric)
mc.compare_max(metric)

mc.history_for(model_name, training_set_name)
```

In [5]:
class ModelComparator:
    def __init__(self):
        self.models = {}
        self.training_sets = {}
        self.histories = {}
    
    def add_model(self, name, model):
        self.models[name] = model
    
    def add_training_set(self, name, data):
        self.training_sets[name] = data
            
    def train_models(self, **kwargs):
        for model_name in self.models:
            for tset_name in self.training_sets:
                print(f"MODEL {model_name} TRAINING ON DATASET {tset_name}")
                
                model = self.models[model_name]()
                
                history = model.fit(
                    self.training_sets[tset_name][0], self.training_sets[tset_name][1],
                    **kwargs
                )

                self.histories[(model_name, tset_name)] = history
                
    def compare_min(self, metric):
        result = {}
        minimum = None
        min_name = None
        for hkey in self.histories:
            history = self.histories[hkey].history
            result[hkey] = np.amin(history[metric])
            if minimum == None or minimum >= result[hkey]:
                minimum = result[hkey]
                min_name = hkey
        return result, {"name" : min_name, "value":minimum}
            
    
    def compare_max(self, metric):
        result = {}
        maximum = None
        max_name = None
        for hkey in self.histories:
            history = self.histories[hkey].history
            result[hkey] = np.amax(history[metric])
            if maximum == None or maximum >= result[hkey]:
                maximum = result[hkey]
                max_name = hkey
        return result, {"name" : max_name, "value": maximum}
    
    def history_for(self, model_name, training_set_name):
        return self.histories[(model_name, training_set_name)]

### Create Models

In [6]:
def nondense_model():
    model_name = "nondense_model"
    
    if not os.path.exists(cb_filepath + "/" + model_name):
        os.makedirs(cb_filepath + "/" + model_name)  
    
    checkpoints = [cb_filepath + '/' + model_name + "/" + name
                   for name in os.listdir(cb_filepath + "/" + model_name)]
    if checkpoints:
        latest_cp = max(checkpoints, key=os.path.getctime )
        print('Restoring from', latest_cp)
        return load_model(latest_cp)
    
    # Input layer of 3 neurons 
    inp = Input(shape=(1,3))
    
    #128 layer
    d2_out = Dense(128)(inp)

    #grab first, 2nd half of the 128 layer
    d2_out_p1 = Lambda(lambda x: x[:,:,0:64])(d2_out)
    d2_out_p2 = Lambda(lambda x: x[:,:,64:128])(d2_out)

    #64 layer(s)
    d3_out = Dense(64)(d2_out_p1)
    d4_out = Dense(64)(d2_out_p2)

    #grab output nodes from both 64 layers
    d5_out = concatenate([d3_out, d4_out])
    
    o = Dense(1)(d5_out)
    
    model = Model(inp, o)
    
    model._name = model_name
    
    model.compile(
        loss="MeanSquaredError",
        metrics=['accuracy']
    )
    
    return model

def dense_model_5L():
    model_name = "dense_model_5L"
    
    if not os.path.exists(cb_filepath + "/" + model_name):
        os.makedirs(cb_filepath + "/" + model_name)  
    
    checkpoints = [cb_filepath + '/' + model_name + "/" + name
                   for name in os.listdir(cb_filepath + "/" + model_name)]
    if checkpoints:
        latest_cp = max(checkpoints, key=os.path.getctime )
        print('Restoring from', latest_cp)
        return load_model(latest_cp)
    
    model_5layer = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1024, input_shape=(1,3)),
        tf.keras.layers.Dense(512),
        tf.keras.layers.Dense(256),
        tf.keras.layers.Dense(128),
        tf.keras.layers.Dense(64),
        tf.keras.layers.Dense(1)
    ])
    
#     model.name = "dense_model"

    model_5layer._name = model_name
    model_5layer.compile(
        loss="MeanSquaredError",
        metrics=['accuracy'] #Acc not working, in testing
    )

    return model_5layer

def dense_model2():
    model_name = "dense_model2"
    
    if not os.path.exists(cb_filepath + "/" + model_name):
        os.makedirs(cb_filepath + "/" + model_name)  
    
    checkpoints = [cb_filepath + '/' + model_name + "/" + name
                   for name in os.listdir(cb_filepath + "/" + model_name)]
    if checkpoints:
        latest_cp = max(checkpoints, key=os.path.getctime )
        print('Restoring from', latest_cp)
        return load_model(latest_cp)
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(2048, input_shape=(1,3)))
    
    model.add(tf.keras.layers.Dense(1024))
    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.Dense(64))
    model.add(tf.keras.layers.Dense(1))
    
#     model.name = "dense_model v2"
    model._name = model_name
    model.compile(
        loss="MeanSquaredError",
        metrics = ["accuracy"]
    )
    
    return model

## Test Model

In [7]:
# def build_model():
#     model = Sequential()

#     model.add(Input(shape=(1,2)))
#     model.add(Dense(64))
#     model.add(Dense(1))

#     model.compile(
#         loss="MeanSquaredError",
#         metrics=['accuracy']
#     )
    
#     return model

# model = build_model()

### Datasets

In [8]:
num_examples = 20000
rstart = 1
rend = 10000

# MULT
mult_setX, mult_setY = ad.gen_data_mult(num_examples, rstart, rend)
mult_setX = mult_setX.reshape(num_examples, 1, 3)

# LOG(MULT)
mult_logX = np.log(mult_setX)
mult_logY = np.log(mult_setY)

### Build ModelComparator

In [9]:
mc = ModelComparator()

mc.add_model("nondense_model", nondense_model)
mc.add_model("dense_model", dense_model_5L)
mc.add_model("dense_model v2", dense_model2)

mc.add_training_set("log normal multiply", (mult_logX, mult_logY))
mc.add_training_set("multiplication", (mult_setX, mult_setY))

## Train Data (Garbage)

In [10]:
# X = np.ones((100,2))
# y = np.ones((100,1))

# model.fit(X, y, epochs=25, batch_size=1, callbacks=[smc])
# model.predict(X)

In [11]:
# model = load_model("checkpoints/chkpt-20.model")

### Train Models

In [13]:
num_epochs = 30
batch_size = num_examples

mc.train_models(
    batch_size=batch_size,
    epochs=num_epochs,
    callbacks=[smc]
)

# new_model = dense_model_5L()
# new_model = load_model("checkpoints/chkpt-200.model")

# new_model.predict(mult_logX)
#new_model

MODEL nondense_model TRAINING ON DATASET log normal multiply
Restoring from checkpoints/nondense_model/900.model
Epoch 1/30
1/1 [==============================] - ETA: 0s - loss: 2707791872.0000 - accuracy: 0.0000e+00INFO:tensorflow:Assets written to: checkpoints/nondense_model/0.model\assets
model saved: checkpoints/nondense_model/0.model
1/1 [==============================] - 2s 2s/step - loss: 2707791872.0000 - accuracy: 0.0000e+00
Epoch 2/30
1/1 [==============================] - 0s 2ms/step - loss: 2629778176.0000 - accuracy: 0.0000e+00
Epoch 3/30
1/1 [==============================] - 0s 998us/step - loss: 2574998272.0000 - accuracy: 0.0000e+00
Epoch 4/30
1/1 [==============================] - 0s 2ms/step - loss: 2530174464.0000 - accuracy: 0.0000e+00
Epoch 5/30
1/1 [==============================] - 0s 9ms/step - loss: 2491131904.0000 - accuracy: 0.0000e+00
Epoch 6/30
1/1 [==============================] - 0s 997us/step - loss: 2455942912.0000 - accuracy: 0.0000e+00
Epoch 7/30
1

1/1 [==============================] - 0s 3ms/step - loss: 192038256.0000 - accuracy: 0.0000e+00
Epoch 6/30
1/1 [==============================] - 0s 3ms/step - loss: 151447952.0000 - accuracy: 0.0000e+00
Epoch 7/30
1/1 [==============================] - 0s 3ms/step - loss: 122533840.0000 - accuracy: 0.0000e+00
Epoch 8/30
1/1 [==============================] - 0s 3ms/step - loss: 101000816.0000 - accuracy: 0.0000e+00
Epoch 9/30
1/1 [==============================] - 0s 3ms/step - loss: 84433976.0000 - accuracy: 0.0000e+00
Epoch 10/30
1/1 [==============================] - 0s 2ms/step - loss: 71366944.0000 - accuracy: 0.0000e+00
Epoch 11/30
1/1 [==============================] - 0s 2ms/step - loss: 60856448.0000 - accuracy: 0.0000e+00
Epoch 12/30
1/1 [==============================] - 0s 2ms/step - loss: 52267820.0000 - accuracy: 0.0000e+00
Epoch 13/30
1/1 [==============================] - 0s 3ms/step - loss: 45158204.0000 - accuracy: 0.0000e+00
Epoch 14/30
1/1 [=======================

KeyboardInterrupt: 

### compare min loss

In [ ]:
models_min_loss, min_loss = mc.compare_min("loss")

In [ ]:
models_min_loss